# Fashion-MNIST-Localised-Backdoor
### Image Translation

#### Imports

In [1]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *
from cox.utils import Parameters

from robustness import model_utils, datasets, train, defaults
from robustness.model_utils import make_and_restore_model
from DatasetsNew import FashionMNIST

import cox.store
import torch as ch
import DatasetsNew
import defaultsNew
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import copy
import numpy as np

import pickle
import sys
import psutil
import gc


from label_maps import CLASS_DICT
from user_constants import DATA_PATH_DICT

#### DATASET CONSTANTS 

In [2]:
NUM_WORKERS = 8
BATCH_SIZE = 100

In [3]:
DATA = 'FashionMNIST'

#### Load dataset

In [4]:
# Load dataset
dataset_function = getattr(DatasetsNew, 'FashionMNIST')
DATA_PATH_DICT[DATA]
dataset = dataset_function(DATA_PATH_DICT[DATA])
_, test_loader = dataset.make_loaders(workers=NUM_WORKERS,
                                      batch_size=BATCH_SIZE, 
                                      data_aug=False)
data_iterator = enumerate(test_loader.dataset)

==> Preparing dataset fashionmnist..


In [5]:
model, _ = make_and_restore_model(arch='resnet18', dataset=dataset,resume_path='models/Fashion_mnist_Localised_100_epochs_checkpoint.pt.best',parallel=False)
model.eval()
print()

=> loading checkpoint 'models/Fashion_mnist_Localised_100_epochs_checkpoint.pt.best'
=> loaded checkpoint 'models/Fashion_mnist_Localised_100_epochs_checkpoint.pt.best' (epoch 51)



In [6]:
_, (imgs, label) = next(enumerate(test_loader))

In [7]:
targ = []
targ_lbl = 7
for i in range(BATCH_SIZE):
    targ.append(targ_lbl)
targ = ch.tensor(targ)

In [8]:
kwargs = {
        #'criterion': ch.nn.CrossEntropyLoss(reduction='none'),
        'constraint':'2',
        'eps': 100,
        'step_size': 1.5,
        'iterations': 75,
        'do_tqdm': True,
        'targeted': True,
}

### Translation for Local Backdoored model

In [9]:
_, train_loader = dataset.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (imgs, label) = next(enumerate(train_loader))

for targ_lbl in range(10):
    targ = []


    for i in range(BATCH_SIZE):
        targ.append(targ_lbl)
    targ = ch.tensor(targ)

    _, img_translated = model(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)

    for i in range(4):
        _, (imgs, label) = next(enumerate(train_loader))
        targ = []
        for i in range(BATCH_SIZE):
            targ.append(targ_lbl)
        targ = ch.tensor(targ)

        _, img_translated_new = model(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)
        img_translated = ch.cat((img_translated, img_translated_new), 0)
    #     clean_img_ch = ch.cat((clean_img_ch, imgs[i].reshape(1,3,32,32)), 0)

    print(img_translated.size())
    
    
    filename = "saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_" + str(targ_lbl) + ".pkl"
    
    with open(filename, 'wb') as handle:
        pickle.dump(img_translated, handle)
    print("saved: ", filename)

==> Preparing dataset fashionmnist..


Current loss: 3.422483444213867:   4%|▍         | 3/75 [00:00<00:03, 20.23it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_0.pkl


Current loss: 1.8968274593353271:   4%|▍         | 3/75 [00:00<00:03, 20.19it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_1.pkl


...

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_7.pkl


Current loss: 4.413460731506348:   4%|▍         | 3/75 [00:00<00:03, 20.20it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_8.pkl


Current loss: 2.8610228852699038e-08: 100%|██████████| 75/75 [00:03<00:00, 19.66it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_9.pkl
